# Parkinson's Disease - Regression Modeling (Predicting UPDRS Scores)
## Stage 3: Training and Evaluating Regression Models

### 🎯 What are we predicting in this notebook?
We are building **regression models** to predict disease severity scores from voice recordings:

**REGRESSION TASK 1: Predict motor_UPDRS**
- **What is motor_UPDRS?** Movement symptom score (scale 0-108)
  - Measures: Tremor, rigidity, slowness, walking ability
  - Higher score = worse movement symptoms
- **Our goal:** Predict this score using only voice features
- **Why?** Allow patients to monitor disease from home without clinic visits

**REGRESSION TASK 2: Predict total_UPDRS**
- **What is total_UPDRS?** Overall disease severity score (scale 0-176)
  - Includes: Motor symptoms + mental state + daily activities
  - Comprehensive measure of disease impact
- **Our goal:** Predict this score from voice recordings
- **Expected:** Harder to predict than motor_UPDRS (includes non-motor symptoms)

### 🔍 How will we evaluate models?
We'll use multiple metrics to compare model performance:

1. **R² Score (R-squared)** - Primary metric
   - Measures: % of variance in UPDRS explained by voice features
   - Range: 0 to 1 (higher is better)
   - Example: R² = 0.87 means model explains 87% of UPDRS variation
   - **Clinical target:** R² > 0.80 (80% accuracy needed for home monitoring)

2. **RMSE (Root Mean Squared Error)**
   - Measures: Average prediction error in UPDRS points
   - Units: Same as UPDRS (points)
   - Example: RMSE = 3.5 means predictions are off by ~3.5 points on average
   - **Clinical target:** RMSE < 4 points (acceptable for monitoring)

3. **MAE (Mean Absolute Error)**
   - Measures: Typical prediction error (less sensitive to outliers than RMSE)
   - Units: UPDRS points
   - Example: MAE = 2.8 means typical error is 2.8 points
   - **Why useful:** Easier to interpret than RMSE

4. **MAPE (Mean Absolute Percentage Error)**
   - Measures: Error as percentage of actual value
   - Example: MAPE = 12% means predictions are typically 12% off
   - **Why useful:** Scale-independent comparison

5. **Residual Analysis**
   - **What:** Difference between predicted and actual values (prediction error)
   - **Why:** Check if errors are random or have patterns
   - **Good model:** Residuals centered at 0, no patterns
   - **Bad model:** Residuals show trends (model is biased)

### 🤖 Models we'll compare (5 total):
1. **Linear Regression** - Baseline (assumes linear relationships)
2. **Polynomial Regression** - Adds interaction terms
3. **Decision Tree** - Handles non-linearity
4. **Random Forest** - Ensemble (expected winner!)
5. **Neural Network** - Deep learning approach

### 📊 What we'll discover:
- Which voice features are most important for prediction?
- Why Random Forest beats Linear Regression (non-linear relationships!)
- Is motor_UPDRS easier to predict than total_UPDRS?
- How much error can we expect (RMSE in UPDRS points)?

In [ ]:
# ============================================================
# SECTION 1: IMPORT LIBRARIES
# ============================================================
# What we're doing: Loading all ML and visualization libraries
# Why: Need scikit-learn for models, PyTorch for neural networks, pandas for data

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Scikit-learn models
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

# Scikit-learn metrics
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

# PyTorch for Neural Network
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

# Set visualization style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

# Set random seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)

print("✅ All libraries imported successfully!")
print(f"PyTorch version: {torch.__version__}")
print(f"Using device: {'CUDA' if torch.cuda.is_available() else 'CPU'}")

In [ ]:
# ============================================================
# SECTION 2: LOAD PREPROCESSED DATA
# ============================================================
# What we're doing: Loading the train/test data we created in preprocessing notebook
# Why: Data is already scaled and split (no need to repeat preprocessing)
#
# Files we're loading:
# - X_train.csv: Training features (4,700 recordings × 26 features)
# - X_test.csv: Test features (1,175 recordings × 26 features)
# - y_train_motor.csv: Training target for motor_UPDRS
# - y_test_motor.csv: Test target for motor_UPDRS
# - y_train_total.csv: Training target for total_UPDRS
# - y_test_total.csv: Test target for total_UPDRS

print("Loading preprocessed data...\\n")

# Load features (X)
X_train = pd.read_csv('../../data/processed/X_train.csv')
X_test = pd.read_csv('../../data/processed/X_test.csv')

# Load motor_UPDRS targets
y_train_motor = pd.read_csv('../../data/processed/y_train_motor.csv').values.ravel()
y_test_motor = pd.read_csv('../../data/processed/y_test_motor.csv').values.ravel()

# Load total_UPDRS targets
y_train_total = pd.read_csv('../../data/processed/y_train_total.csv').values.ravel()
y_test_total = pd.read_csv('../../data/processed/y_test_total.csv').values.ravel()

print("✅ Data loaded successfully!\\n")
print("="*60)
print("📊 DATASET SUMMARY:")
print("="*60)
print(f"\\nTraining set:")
print(f"  X_train shape:      {X_train.shape} (recordings × features)")
print(f"  y_train_motor:      {y_train_motor.shape} values")
print(f"  y_train_total:      {y_train_total.shape} values")
print(f"\\nTest set:")
print(f"  X_test shape:       {X_test.shape}")
print(f"  y_test_motor:       {y_test_motor.shape} values")
print(f"  y_test_total:       {y_test_total.shape} values")
print(f"\\nFeatures: {X_train.shape[1]} (all scaled with StandardScaler)")
print(f"\\n💡 Note: Same X features used for both motor and total UPDRS prediction")
print(f"   Only the target variable (y) changes!")

---
## 🎯 PART 1: PREDICTING motor_UPDRS (Movement Symptoms)

### What we're predicting:
- **Target:** motor_UPDRS score (0-108 scale)
- **Meaning:** How severe are the patient's movement symptoms?
  - Low score (~10-15): Mild tremor, slight stiffness
  - Medium score (~20-30): Noticeable movement problems
  - High score (~35-40): Severe motor impairment
- **Input:** 26 voice features (jitter, shimmer, noise, complexity, age, test_time, etc.)
- **Goal:** Build models that can predict this score from voice alone

### Why this matters clinically:
If we can predict motor_UPDRS accurately (R² > 0.80), patients can:
- Record voice at home daily
- Get instant motor symptom assessment
- Detect deterioration early
- Avoid weekly clinic visits ($200-500 per visit)

In [ ]:
# ============================================================
# MODEL 1: LINEAR REGRESSION (Baseline Model)
# ============================================================
# What we're doing: Training a simple linear regression model
# 
# How it works:
#   motor_UPDRS = β₀ + β₁(Jitter) + β₂(Shimmer) + ... + β₂₆(feature₂₆)
#
# Assumptions:
#   - Linear relationship between voice features and UPDRS
#   - Each feature has constant effect (no interactions)
#   - Example: "1 unit increase in Jitter → always adds 2.5 points to UPDRS"
#
# Expected performance:
#   - Decent baseline (R² ~0.70-0.75)
#   - Won't capture non-linear patterns
#   - Fast to train, easy to interpret
#
# Why it won't win:
#   - Real relationship is NON-LINEAR
#   - Misses feature interactions (e.g., high Jitter + high Shimmer is worse than sum)

print("\\n" + "="*60)
print("🤖 MODEL 1: LINEAR REGRESSION")
print("="*60)

# Initialize model
lr_model = LinearRegression()

# Train on training data
print("\\nTraining Linear Regression...")
lr_model.fit(X_train, y_train_motor)

# Make predictions
y_train_pred_lr = lr_model.predict(X_train)
y_test_pred_lr = lr_model.predict(X_test)

print("✅ Training complete!")
print(f"\\nModel learned {X_train.shape[1]} coefficients (one per feature)")
print(f"Intercept: {lr_model.intercept_:.3f}")

In [ ]:
# ============================================================
# EVALUATION BLOCK: LINEAR REGRESSION PERFORMANCE
# ============================================================
# What we're evaluating: How well does Linear Regression predict motor_UPDRS?
#
# Metrics we're calculating:
# 1. R² (R-squared): % of variance explained
# 2. RMSE (Root Mean Squared Error): Average error in UPDRS points
# 3. MAE (Mean Absolute Error): Typical error magnitude
# 4. MAPE (Mean Absolute Percentage Error): Error as percentage
#
# We calculate these for BOTH train and test:
# - Train metrics: How well model fits training data
# - Test metrics: How well model generalizes to new data
# - Gap between train/test: Indicates overfitting
#   * Small gap (< 5%): Good generalization
#   * Large gap (> 15%): Model overfitting to training data

print("\\n" + "-"*60)
print("📊 EVALUATION: LINEAR REGRESSION on motor_UPDRS")
print("-"*60)

# Calculate metrics for TRAINING set
train_r2_lr = r2_score(y_train_motor, y_train_pred_lr)
train_rmse_lr = np.sqrt(mean_squared_error(y_train_motor, y_train_pred_lr))
train_mae_lr = mean_absolute_error(y_train_motor, y_train_pred_lr)
train_mape_lr = np.mean(np.abs((y_train_motor - y_train_pred_lr) / y_train_motor)) * 100

# Calculate metrics for TEST set
test_r2_lr = r2_score(y_test_motor, y_test_pred_lr)
test_rmse_lr = np.sqrt(mean_squared_error(y_test_motor, y_test_pred_lr))
test_mae_lr = mean_absolute_error(y_test_motor, y_test_pred_lr)
test_mape_lr = np.mean(np.abs((y_test_motor - y_test_pred_lr) / y_test_motor)) * 100

print("\\n📈 TRAINING SET PERFORMANCE:")
print(f"  R² Score:  {train_r2_lr:.4f}  → Explains {train_r2_lr*100:.2f}% of motor_UPDRS variance")
print(f"  RMSE:      {train_rmse_lr:.3f} points → Average error magnitude")
print(f"  MAE:       {train_mae_lr:.3f} points → Typical error (less sensitive to outliers)")
print(f"  MAPE:      {train_mape_lr:.2f}% → Average percentage error")

print("\\n📉 TEST SET PERFORMANCE (What really matters!):")
print(f"  R² Score:  {test_r2_lr:.4f}  → Explains {test_r2_lr*100:.2f}% of motor_UPDRS variance")
print(f"  RMSE:      {test_rmse_lr:.3f} points → Predicting within ~{test_rmse_lr:.1f} UPDRS points")
print(f"  MAE:       {test_mae_lr:.3f} points → Typical prediction error")
print(f"  MAPE:      {test_mape_lr:.2f}% → Average percentage error")

# Calculate overfitting gap
r2_gap = (train_r2_lr - test_r2_lr) * 100
rmse_gap = ((test_rmse_lr - train_rmse_lr) / train_rmse_lr) * 100

print("\\n⚖️  OVERFITTING CHECK (Train vs Test):")
print(f"  R² gap:    {r2_gap:.2f}% {'✅ Good!' if r2_gap < 5 else '⚠️ Some overfitting' if r2_gap < 15 else '❌ Overfitting!'}")
print(f"  RMSE increase: {rmse_gap:.2f}% on test set")

print("\\n💡 INTERPRETATION:")
if test_r2_lr > 0.80:
    print(f"  ✅ Excellent! R² > 0.80 means model is clinically useful")
elif test_r2_lr > 0.70:
    print(f"  👍 Good baseline! R² = {test_r2_lr:.2f} is decent but can improve")
else:
    print(f"  ⚠️  Weak performance. R² = {test_r2_lr:.2f} means model misses too much variance")

print(f"\\n  Clinical meaning: Predictions are typically off by {test_mae_lr:.1f} UPDRS points")
print(f"  For a patient with motor_UPDRS = 25, we'd predict {25-test_mae_lr:.1f} to {25+test_mae_lr:.1f}")

In [ ]:
# ============================================================
# VISUALIZATION: LINEAR REGRESSION - PREDICTED VS ACTUAL
# ============================================================
# What we're visualizing: How close are predictions to actual values?
#
# Perfect model: All points on diagonal line (predicted = actual)
# Good model: Points clustered around diagonal
# Bad model: Points scattered far from diagonal
#
# What to look for:
# - Points above line: Model UNDERpredicts (says UPDRS is lower than reality)
# - Points below line: Model OVERpredicts (says UPDRS is higher than reality)
# - Spread: How much error/uncertainty

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Training set predictions
axes[0].scatter(y_train_motor, y_train_pred_lr, alpha=0.5, s=20, color='blue', edgecolor='black', linewidth=0.5)
axes[0].plot([y_train_motor.min(), y_train_motor.max()], [y_train_motor.min(), y_train_motor.max()], 
             'r--', linewidth=2, label='Perfect Prediction')
axes[0].set_xlabel('Actual motor_UPDRS', fontsize=11)
axes[0].set_ylabel('Predicted motor_UPDRS', fontsize=11)
axes[0].set_title(f'Linear Regression: Training Set\\nR² = {train_r2_lr:.3f}, RMSE = {train_rmse_lr:.2f}', 
                  fontweight='bold')
axes[0].legend()
axes[0].grid(alpha=0.3)

# Test set predictions
axes[1].scatter(y_test_motor, y_test_pred_lr, alpha=0.5, s=20, color='green', edgecolor='black', linewidth=0.5)
axes[1].plot([y_test_motor.min(), y_test_motor.max()], [y_test_motor.min(), y_test_motor.max()], 
             'r--', linewidth=2, label='Perfect Prediction')
axes[1].set_xlabel('Actual motor_UPDRS', fontsize=11)
axes[1].set_ylabel('Predicted motor_UPDRS', fontsize=11)
axes[1].set_title(f'Linear Regression: Test Set\\nR² = {test_r2_lr:.3f}, RMSE = {test_rmse_lr:.2f}', 
                  fontweight='bold')
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("💡 How to read this plot:")
print("  - Red dashed line = perfect predictions")
print("  - Points on the line = model predicted exactly right")
print("  - Points above line = underprediction (model says lower UPDRS than actual)")
print("  - Points below line = overprediction (model says higher UPDRS than actual)")
print("  - Closer to line = better model!")

In [ ]:
# ============================================================
# VISUALIZATION: RESIDUAL ANALYSIS (Prediction Errors)
# ============================================================
# What are residuals? residual = actual - predicted (the error we made)
#
# Why analyze residuals?
# - Check if errors are random or have patterns
# - Good model: Residuals centered at 0, no trends
# - Bad model: Residuals show patterns (model is biased)
#
# What we're looking for:
# 1. Centered at zero: Mean residual should be ~0 (no systematic bias)
# 2. Constant spread: Variance shouldn't change with predicted value (homoscedasticity)
# 3. Normal distribution: Residuals should be bell-shaped
# 4. No patterns: Random scatter (no trends, no curves)

# Calculate residuals
train_residuals_lr = y_train_motor - y_train_pred_lr
test_residuals_lr = y_test_motor - y_test_pred_lr

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Residuals vs Predicted (Training)
axes[0, 0].scatter(y_train_pred_lr, train_residuals_lr, alpha=0.5, s=20, color='blue')
axes[0, 0].axhline(0, color='red', linestyle='--', linewidth=2, label='Zero Error')
axes[0, 0].set_xlabel('Predicted motor_UPDRS')
axes[0, 0].set_ylabel('Residual (Actual - Predicted)')
axes[0, 0].set_title('Residual Plot: Training Set\\n(Should be random scatter around 0)', fontweight='bold')
axes[0, 0].legend()
axes[0, 0].grid(alpha=0.3)

# 2. Residuals vs Predicted (Test)
axes[0, 1].scatter(y_test_pred_lr, test_residuals_lr, alpha=0.5, s=20, color='green')
axes[0, 1].axhline(0, color='red', linestyle='--', linewidth=2, label='Zero Error')
axes[0, 1].set_xlabel('Predicted motor_UPDRS')
axes[0, 1].set_ylabel('Residual (Actual - Predicted)')
axes[0, 1].set_title('Residual Plot: Test Set\\n(Check for patterns or trends)', fontweight='bold')
axes[0, 1].legend()
axes[0, 1].grid(alpha=0.3)

# 3. Residual distribution histogram (Training)
axes[1, 0].hist(train_residuals_lr, bins=30, color='blue', edgecolor='black', alpha=0.7)
axes[1, 0].axvline(train_residuals_lr.mean(), color='red', linestyle='--', linewidth=2, 
                   label=f'Mean = {train_residuals_lr.mean():.3f}')
axes[1, 0].set_xlabel('Residual Value')
axes[1, 0].set_ylabel('Frequency')
axes[1, 0].set_title(f'Residual Distribution: Training\\n(Should be normal, mean ≈ 0)', fontweight='bold')
axes[1, 0].legend()
axes[1, 0].grid(alpha=0.3)

# 4. Residual distribution histogram (Test)
axes[1, 1].hist(test_residuals_lr, bins=30, color='green', edgecolor='black', alpha=0.7)
axes[1, 1].axvline(test_residuals_lr.mean(), color='red', linestyle='--', linewidth=2, 
                   label=f'Mean = {test_residuals_lr.mean():.3f}')
axes[1, 1].set_xlabel('Residual Value')
axes[1, 1].set_ylabel('Frequency')
axes[1, 1].set_title(f'Residual Distribution: Test\\n(Should be normal, mean ≈ 0)', fontweight='bold')
axes[1, 1].legend()
axes[1, 1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("\\n📊 RESIDUAL ANALYSIS SUMMARY:")
print("="*60)
print(f"Training set residuals: Mean = {train_residuals_lr.mean():.4f}, Std = {train_residuals_lr.std():.3f}")
print(f"Test set residuals:     Mean = {test_residuals_lr.mean():.4f}, Std = {test_residuals_lr.std():.3f}")
print("\\n💡 What to look for:")
print("  ✅ Mean ≈ 0: No systematic bias (not consistently over/under predicting)")
print("  ✅ Random scatter: No patterns in residual plots (model captured all relationships)")
print("  ✅ Normal distribution: Bell-shaped histogram (assumptions valid)")
print("  ✅ Constant spread: Similar variance across all predicted values")

if abs(test_residuals_lr.mean()) < 1.0:
    print("\\n✅ Residuals are well-centered (mean near 0)")
else:
    print(f"\\n⚠️  Slight bias detected (mean = {test_residuals_lr.mean():.2f})")

In [ ]:
# ============================================================
# MODEL 2: RANDOM FOREST REGRESSION (Expected Winner!)
# ============================================================
# What we're doing: Training an ensemble of 100 decision trees
#
# How Random Forest works:
# 1. Create 100 different decision trees
# 2. Each tree trained on random subset of data (bootstrap sampling)
# 3. Each tree considers random subset of features at each split
# 4. Final prediction = average of all 100 tree predictions
#
# Why it's better than Linear Regression:
# - Captures NON-LINEAR relationships (e.g., "Jitter matters MORE when Shimmer is high")
# - Handles INTERACTIONS automatically (e.g., Jitter × Shimmer)
# - Robust to OUTLIERS (averaging reduces their impact)
# - Prevents OVERFITTING (bagging decorrelates trees)
#
# Expected performance:
# - R² ~0.85-0.90 (much better than Linear Regression's ~0.73)
# - RMSE ~2.5-3.5 points (vs Linear's ~4.0)
# - Small train/test gap (good generalization)
#
# Bonus: Feature importance
# - Tells us which voice features are most important
# - Expected top features: HNR, Jitter(%), Shimmer

print("\\n" + "="*60)
print("🤖 MODEL 2: RANDOM FOREST REGRESSION")
print("="*60)

# Initialize Random Forest
rf_model = RandomForestRegressor(
    n_estimators=100,        # 100 trees in the forest
    max_depth=15,           # Maximum tree depth (prevent overfitting)
    min_samples_split=20,   # Need 20 samples to split a node
    min_samples_leaf=10,    # Minimum 10 samples in each leaf
    max_features='sqrt',    # Consider √26 ≈ 5 random features per split
    random_state=42,        # Reproducibility
    n_jobs=-1               # Use all CPU cores
)

# Train the model
print("\\nTraining Random Forest (100 trees)...")
print("This may take a few seconds...")
rf_model.fit(X_train, y_train_motor)

# Make predictions
y_train_pred_rf = rf_model.predict(X_train)
y_test_pred_rf = rf_model.predict(X_test)

print("✅ Training complete!")
print(f"\\nModel configuration:")
print(f"  - Number of trees: {rf_model.n_estimators}")
print(f"  - Max tree depth: {rf_model.max_depth}")
print(f"  - Features per split: {rf_model.max_features}")
print(f"  - Total features used: {X_train.shape[1]}")